# PyACP performance test with slower network

This notebook tests the PyACP performance when run with a slower network connection. It uses the Linux ``tc-netem`` command to emulate a slower (latency- and bandwidth-restricted) network in a Docker container.

Note that not all Linux kernels support ``tc-netem``: For example, it cannot be run on WSL2.

## Build docker image

In [ ]:
!DOCKER_BUILDKIT=1 docker build --tag pyacp-connectiontest -f ../dockerfiles/ConnectionTest.Dockerfile .

## Launch servers

In [ ]:
import ansys.acp.core as pyacp

In [ ]:
slow_launch = pyacp.DockerComposeLaunchConfig(
    image_name_pyacp="pyacp-connectiontest",
    compose_file="../docker-compose/docker-compose-connectiontest.yaml",
    environment_variables={"PYACP_DELAY": "200ms", "PYACP_RATE": "1mbit"},
)
normal_launch = pyacp.DockerComposeLaunchConfig()


In [ ]:
slow_server = pyacp.launch_acp(config=slow_launch)
normal_server = pyacp.launch_acp(config=normal_launch)

In [ ]:
slow_server.wait(timeout=30)
normal_server.wait(timeout=30)

In [ ]:
client_slow = pyacp.Client(slow_server)
client_normal = pyacp.Client(normal_server)

## Run example

In [ ]:
def run_example(client):
    f = client.upload_file(local_path="../examples/data/class40/class40.cdb")
    m = client.import_model(path=f, format="ansys:cdb")
    mg = m.create_modeling_group()
    for _ in range(100):
        mg.create_modeling_ply()
    client.clear()

In [ ]:
run_example(client_normal)

In [ ]:
run_example(client_slow)